In [1]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

If the dataset is in a zipped,it can be unzipped by using the ZipFile module.There are other options such as unzip <filename>.zip for linux and macos.
The dataset i'm using is readily unzipped to use.
 
For those using google colab,it is advised to upload the dataset to google drive and mount the drive to your colab.Note that it takes a lot of times if your dataset is anywhere between medium to Large.
 
For kaggle users,Datasets from kaggle can be added by using the add dataset option

In [2]:
# import zipfile
# with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip','r') as file:
#     file.extractall('.')
#     print('unzipped')

To label the images,we will be using flow_from_directory method,which requires the data be in a specific heirarchy<br>
**/mainDirectory<br>
  ----/label1<br>
        ---------/data1<br>
        ---------/data2<br>
  ----/label2<br>
        --------/data1<br>
        --------/data2<br>**
 Since all our data is conveniently named with either a cat or dog as the starting part of the name,sorting into a heirarchy is much easier.

In [3]:
# import cv2,os,shutil
# datasetdir = '/kaggle/working/train'
# cat_dir = os.path.join(datasetdir, 'cats')
# dog_dir = os.path.join(datasetdir, 'dogs')
# os.makedirs(cat_dir, exist_ok=True)
# os.makedirs(dog_dir, exist_ok=True)
# for filename in os.listdir(datasetdir):
#     if filename.startswith('cat'):
#         src_path = os.path.join(datasetdir, filename)
#         shutil.move(src_path, cat_dir)
# #         print(f"Moved {filename} to cats directory")
#     elif filename.startswith('dog'):
#         src_path = os.path.join(datasetdir, filename)
#         shutil.move(src_path, dog_dir)
# #         print(f"Moved {filename} to dogs directory")
    

As a rule of the thumb, 20% of the data will be reserved for validation.
ImageDataGen is a data generator for the training data.
* Flow_from_directory method reads and preprocesses the images from the specified directory.
* Target_size (hyperparameter) resizes the images to a size of (200, 200) pixels. 
* Batch_size (hyperparameter) determines how many images are included in each batch during training.This batch_size is highly proportional to the learning_rate. 
* Classes specifies the class labels for the dataset ('dogs' and 'cats'). 
* The subset parameter is set to 'training' to indicate that this generator is for the training subset of data

In [4]:
import tensorflow as tf
datasetdir = '/kaggle/input/cat-and-dog/training_set/training_set'
ImageDataGen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split= 0.2)
train_generator = ImageDataGen.flow_from_directory(directory = datasetdir, target_size=(200,200), batch_size=25, classes = ('dogs','cats'), subset= 'training')
val_generator = ImageDataGen.flow_from_directory(directory = datasetdir, target_size=(200,200), batch_size=25, classes = ('dogs','cats'), subset= 'validation')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


* Sequential model is a linear stack of layers.

* This architecture has three sets of convolutional layers, each followed by batch normalization, max pooling, and dropout layers. These layers help the model learn hierarchical features from the input images.

* Flattening converts the output into 1D array.

* Two dense layers are added with ReLU activation, batch normalization, and dropout. The first dense layer has 512 units, and the second dense layer has 2 units with a softmax activation(function that scales numbers/logits into probabilities).


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(200,200,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1,activation='softmax'))

model.compile(loss='binary_crossentropy',
  optimizer='adam',metrics=['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 198, 198, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 99, 99, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 97, 97, 64)       2

* EarlyStopping is a callback that stops training if a metric (such as validation loss or accuracy) reamins at a saturated level for a certain number of epochs (here 10)
* ReduceLROnPlateau is another callback that slowly reduces the learning rate when the aforementioned saturation is observed

In [6]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.01)
callbacks = [earlystop,learning_rate_reduction]

* Epoch specifies the number of times the model will iterate over the dataset


In [7]:
history = model.fit(
    train_generator, 
    validation_data = val_generator,
    workers=10,
    epochs=50,
)

Epoch 1/50


2023-08-10 19:31:34.389509: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


257/257 [==============================] - 32s 81ms/step - loss: 0.7412 - accuracy: 0.5000 - val_loss: 1.7185 - val_accuracy: 0.5000
Epoch 2/50
257/257 [==============================] - 21s 82ms/step - loss: 0.7012 - accuracy: 0.5000 - val_loss: 2.8022 - val_accuracy: 0.5000
Epoch 3/50
257/257 [==============================] - 21s 82ms/step - loss: 0.6960 - accuracy: 0.5000 - val_loss: 4.1602 - val_accuracy: 0.5000
Epoch 4/50
257/257 [==============================] - 21s 82ms/step - loss: 0.6942 - accuracy: 0.5000 - val_loss: 5.5312 - val_accuracy: 0.5000
Epoch 5/50
257/257 [==============================] - 21s 81ms/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 6.0302 - val_accuracy: 0.5000
Epoch 6/50
257/257 [==============================] - 22s 83ms/step - loss: 0.7052 - accuracy: 0.5000 - val_loss: 0.7585 - val_accuracy: 0.5000
Epoch 7/50
257/257 [==============================] - 22s 85ms/step - loss: 0.6944 - accuracy: 0.5000 - val_loss: 0.7156 - val_accuracy: 0.5000
Epo

In [8]:
evaluation = model.evaluate(val_generator)
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])


65/65 [==============================] - 4s 55ms/step - loss: 0.6968 - accuracy: 0.5000
Validation Loss: 0.6967554092407227
Validation Accuracy: 0.5


In [9]:
model.save("model1_catsVSdogs_10epoch.h5")